In [31]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import LabelBinarizer

In [166]:
import statsmodels.formula.api as smf

In [303]:
ORIGINAL = pd.read_csv('data/survey_results_public_mega_inc.csv')

/home/jvanderwolf/projects/school/codam/events/no-bias-no-cry/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning:

Columns (11,12,13,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.



In [304]:
df = ORIGINAL.copy()

In [305]:
VARS_REQUIRED = [
    'ConvertedSalary',
    'JobSatisfaction',
    'CareerSatisfaction',
    'SexualOrientation',
    'DevType',
    'EducationTypes',
    'YearsCoding',
    'YearsCodingProf',
    'Gender'
]

In [306]:
# big filters
df_filter_country = df[(df['Country'] == 'Netherlands') | (df['Country'] == 'Germany')]
df_filter_employ = df_filter_country[(df_filter_country['Employment'] == 'Employed full-time')]
df_filter_sex = df_filter_employ[(df_filter_employ['Gender'] == 'Male') | (df_filter_employ['Gender'] == 'Female')]
df_filter_salary = df_filter_sex[VARS_REQUIRED]

In [308]:
# clean up and dummy data replacements
df_filter_salary = df_filter_salary[(df_filter_salary['Gender'].notnull()) & (df_filter_salary['SexualOrientation'].notnull()) & (df_filter_salary['JobSatisfaction'].notnull())]
df_filter_salary['JobSatisfaction'] = df_filter_salary['JobSatisfaction'].replace({
    'Extremely dissatisfied': 0,
    'Moderately dissatisfied': 1,
    'Slightly dissatisfied': 2,
    'Neither satisfied nor dissatisfied': 3,
    'Slightly satisfied': 4,
    'Moderately satisfied': 5,
    'Extremely satisfied': 6
})

In [309]:
# clean up np.NaN values
df_filter_salary['GenderBinary'] = np.where((df_filter_salary['Gender'] == 'Male'), 1, 0)
df_filter_salary['OrientationBinary'] = np.where((df_filter_salary['SexualOrientation'] == 'Straight or heterosexual'), 'OrientConforming', 'OrientNonConforming')

In [310]:
filtered = df_filter_salary.copy()

In [311]:
filtered = pd.concat([filtered, pd.get_dummies(filtered['OrientationBinary'])], axis=1)
filtered = pd.concat([filtered, pd.get_dummies(filtered['SexualOrientation'])], axis=1)

In [316]:
final_df = filtered[['ConvertedSalary', 'OrientationBinary', 'YearsCodingProf', 'GenderBinary', 'JobSatisfaction']]
results1 = smf.ols('ConvertedSalary ~ OrientationBinary + YearsCodingProf + GenderBinary', data=final_df).fit()

In [317]:
results1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        ConvertedSalary   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     4.746
Date:                Fri, 05 Mar 2021   Prob (F-statistic):           9.77e-08
Time:                        17:17:51   Log-Likelihood:                -38518.
No. Observations:                2869   AIC:                         7.706e+04
Df Residuals:                    2856   BIC:                         7.714e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                 7.149e+04   1.58e+04      4.517      0.000    4.05e+04    1.03e+05
OrientationBinary[T.OrientNonConforming] -1.523e+04   1.34e+04     -1.141      0.254   -4.14e+04     1.1e+04
YearsCodingProf[T.12-14 years]            4.353e+04   1.37e+04      3.186      0.001    1.67e+04    7.03e+04
YearsCodingProf[T.15-17 years]            7.133e+04   1.63e+04      4.379      0.000    3.94e+04    1.03e+05
YearsCodingProf[T.18-20 years]            5.396e+04   1.76e+04      3.068      0.002    1.95e+04    8.85e+04
YearsCodingProf[T.21-23 years]            4.274e+04   2.62e+04      1.633      0.103   -8581.009    9.41e+04
YearsCodingProf[T.24-26 years]            8.935e+04   2.96e+04      3.019      0.003    3.13e+04    1.47e+05
YearsCodingProf[T.27-29 years]             8.16e+04   4.62e+04      1.767      0.077   -8953.302    1.72e+05
YearsCodingProf[T.3-5 years]              1.993e+04   9358.043      2.130      0.033    1584.035    3.83e+04
YearsCodingProf[T.30 or more years]        9.82e+04   3.44e+04      2.857      0.004    3.08e+04    1.66e+05
YearsCodingProf[T.6-8 years]              3.053e+04   1.03e+04      2.974      0.003    1.04e+04    5.07e+04
YearsCodingProf[T.9-11 years]               6.2e+04   1.13e+04      5.479      0.000    3.98e+04    8.42e+04
GenderBinary                             -2611.6912   1.49e+04     -0.175      0.861   -3.19e+04    2.66e+04
==============================================================================
Omnibus:                     2508.640   Durbin-Watson:                   2.061
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            49194.212
Skew:                           4.365   Prob(JB):                         0.00
Kurtosis:                      21.312   Cond. No.                         22.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [318]:
results2 = smf.ols('JobSatisfaction ~ OrientationBinary + YearsCodingProf + GenderBinary', data=final_df).fit()

In [319]:
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        JobSatisfaction   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.764
Date:                Fri, 05 Mar 2021   Prob (F-statistic):           0.000947
Time:                        17:17:58   Log-Likelihood:                -6793.6
No. Observations:                3488   AIC:                         1.361e+04
Df Residuals:                    3475   BIC:                         1.369e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    4.1829      0.148     28.289      0.000       3.893       4.473
OrientationBinary[T.OrientNonConforming]    -0.0184      0.124     -0.148      0.882      -0.261       0.225
YearsCodingProf[T.12-14 years]              -0.4487      0.127     -3.536      0.000      -0.698      -0.200
YearsCodingProf[T.15-17 years]              -0.0333      0.147     -0.227      0.820      -0.321       0.254
YearsCodingProf[T.18-20 years]              -0.4295      0.164     -2.625      0.009      -0.750      -0.109
YearsCodingProf[T.21-23 years]              -0.2720      0.228     -1.191      0.234      -0.720       0.176
YearsCodingProf[T.24-26 years]              -0.1940      0.268     -0.723      0.470      -0.720       0.332
YearsCodingProf[T.27-29 years]              -0.6033      0.418     -1.443      0.149      -1.423       0.216
YearsCodingProf[T.3-5 years]                -0.3209      0.086     -3.712      0.000      -0.490      -0.151
YearsCodingProf[T.30 or more years]          0.1765      0.323      0.546      0.585      -0.457       0.810
YearsCodingProf[T.6-8 years]                -0.3776      0.095     -3.972      0.000      -0.564      -0.191
YearsCodingProf[T.9-11 years]               -0.3618      0.105     -3.445      0.001      -0.568      -0.156
GenderBinary                                 0.2449      0.139      1.766      0.077      -0.027       0.517
==============================================================================
Omnibus:                      396.428   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              527.477
Skew:                          -0.940   Prob(JB):                    2.88e-115
Kurtosis:                       2.695   Cond. No.                         21.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [259]:
# DevType
# EducationTypes
# Gender
# Orientation
uniques = []
columns = ['DevType', 'EducationTypes', 'Gender', 'SexualOrientation']
for col in columns:
    options = filtered[col]
    options = options.replace(np.nan, 'n/a')
    for s in options:
        if s.find(';'):
            split = s.split(';')
            for _s in split:
                if _s not in uniques:
                    uniques.append(_s)
        else:
            if s not in uniques:
                uniques.append(s)


In [260]:
uniques

['Back-end developer',
 'Desktop or enterprise applications developer',
 'Embedded applications or devices developer',
 'Front-end developer',
 'Full-stack developer',
 'System administrator',
 'n/a',
 'Database administrator',
 'Mobile developer',
 'QA or test developer',
 'C-suite executive (CEO, CTO, etc.)',
 'DevOps specialist',
 'Engineering manager',
 'Data scientist or machine learning specialist',
 'Educator or academic researcher',
 'Student',
 'Designer',
 'Data or business analyst',
 'Marketing or sales professional',
 'Game or graphics developer',
 'Product manager',
 'Completed an industry certification program (e.g. MCPD)',
 'Received on-the-job training in software development',
 'Taught yourself a new language, framework, or tool without taking a formal course',
 'Contributed to open source software',
 'Taken an online course in programming or software development (e.g. a MOOC)',
 'Taken a part-time in-person course in programming or software development',
 'Participate